## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd
import tensorflow as tf
import operator

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=["EIN","NAME"])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [5]:
# Verify that the number of columns has changed after the drop
# Original data
print(len(application_df.columns))
# dropped-column data
print(len(df.columns))

12
10


In [6]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning

# One syntax
# print(df.value_counts("APPLICATION_TYPE"))

# Alternate syntax
print(df["APPLICATION_TYPE"].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [8]:
def find_df_percentage(dataframe, column, operator_keep, operator_cutoff, cutoff):
  """
  Create two lists of application types and their value_counts (frequency) in the dataframe
  One list for those whose value is above cutoff
  One list for those whose value is below cutoff

  """
  df = dataframe
  cutoff_value = cutoff

  # Confirming info when using the function later
  print(f"The '{column}' column will be filtered by {cutoff}")
  print("")

  application_types_keep = [(index, value) for index, value in df[column].value_counts().items() if operator_keep(value, cutoff_value)]
  application_types_cutoff = [(index, value) for index, value in df[column].value_counts().items() if operator_cutoff(value, cutoff_value)]

  # Print statements for verification of list-comprehensions above
  # print(application_types_keep)
  # print(application_types_cutoff)

  # Get the sums of rows kept and cutoff, then the total of the sums.
  data_rows_kept = sum([value for index, value in application_types_keep])
  data_rows_cutoff = sum([value for index, value in application_types_cutoff])
  data_rows_total = data_rows_kept + data_rows_cutoff

  # Test if the total rows == the dataset they're filtering
  total_check = data_rows_total == len(df)
  print(f"It is {total_check} that there was no data lost when filtering the dataframe.")
  print("")

  def calculate_percentage(part, whole):
    return (part / whole) * 100

  # Calculate the percentage of each of the sums to the entire dataset.
  percentage_of_kept_data = calculate_percentage(data_rows_kept, data_rows_total)
  percentage_of_cutoff_data = calculate_percentage(data_rows_cutoff, data_rows_total)

  print(f"{data_rows_kept} rows of the data were kept, which is about {round(percentage_of_kept_data, 2)} of the data full dataset.")
  print(f"{data_rows_cutoff} rows of the data were cutoff, which is about {round(percentage_of_cutoff_data, 2)} of the data full dataset.")

  # return raw percentage (float) values
  return percentage_of_kept_data, percentage_of_cutoff_data

# Application_type value_counts

In [9]:
# Calling function to estimate the percentage of the original dataset that a cutoff_value of 50 would remove

# Operators initialized into variables for better readibility
greater_than_equal = operator.ge
less_than = operator.lt
cutoff_value = 500

# run function
perc_kept, perc_cutoff = find_df_percentage(dataframe= df,
                   column= "APPLICATION_TYPE",
                   operator_keep= greater_than_equal,
                   operator_cutoff= less_than,
                   cutoff= cutoff_value)

# Using the floats returned by the function
print("")
print("Raw Percentages:")
print(f"{perc_kept} kept")
print(f"{perc_cutoff} cutoff")

The 'APPLICATION_TYPE' column will be filtered by 500

It is True that there was no data lost when filtering the dataframe.

34023 rows of the data were kept, which is about 99.2 of the data full dataset.
276 rows of the data were cutoff, which is about 0.8 of the data full dataset.

Raw Percentages:
99.19531181667104 kept
0.8046881833289601 cutoff


In [10]:
# Choose a cutoff value and create a list of application types to be replaced and use the variable name `application_types_to_replace`

# Value chosen as 500 (verified in cell above)
# list comprehension below selects only the application types
application_types_cutoff = [(index, value) for index, value in df['APPLICATION_TYPE'].value_counts().items() if value < 500]
application_types_to_replace = [index for index, value in application_types_cutoff]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

# Classification value_counts

In [11]:
# Look at CLASSIFICATION value counts for binning
print(df["CLASSIFICATION"].value_counts()[:10])

# Visualized (uncomment)
# df["CLASSIFICATION"].value_counts().plot(kind="line")

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64


In [12]:
# You may find it helpful to look at CLASSIFICATION value counts > 1

# Calling function to estimate the percentage of the original dataset that a cutoff_value of 50 would remove

# Operators initialized into variables for better readibility
greater_than = operator.gt
less_than_equal = operator.le
cutoff_value = 500

# run function
perc_kept, perc_cutoff = find_df_percentage(dataframe= df,
                   column= "CLASSIFICATION",
                   operator_keep= greater_than,
                   operator_cutoff= less_than_equal,
                   cutoff= cutoff_value)

print("")
print("Raw Percentages:")
print(f"{perc_kept} kept")
print(f"{perc_cutoff} cutoff")

The 'CLASSIFICATION' column will be filtered by 500

It is True that there was no data lost when filtering the dataframe.

32815 rows of the data were kept, which is about 95.67 of the data full dataset.
1484 rows of the data were cutoff, which is about 4.33 of the data full dataset.

Raw Percentages:
95.67334324615878 kept
4.3266567538412195 cutoff


In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classification_types_to_replace`

cutoff_value = 1000
classification_types_cutoff = [(index, value) for index, value in df["CLASSIFICATION"].value_counts().items() if value < cutoff_value]
classification_types_to_replace = [index for index, value in classification_types_cutoff]

# Replace in dataframe
for cls in classification_types_to_replace:
    df["CLASSIFICATION"] = df["CLASSIFICATION"].replace(cls,"Other")

# Check to make sure binning was successful
df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [15]:
# Split our preprocessed data into our features and target arrays

X = df_encoded.drop(columns="IS_SUCCESSFUL").values
y = df_encoded["IS_SUCCESSFUL"].values

# Shape targets
y = y.reshape(-1, 1)


# # Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)


In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=3, activation="tanh", input_dim=36))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 3)                 111       
                                                                 
 dense_4 (Dense)             (None, 4)                 16        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 132 (528.00 Byte)
Trainable params: 132 (528.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
# import the ModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='model_weights.h5',  # Filepath to save the model weights
    save_weights_only=True,  # Only save the model weights, not the entire model
    save_best_only=False,  # Save the weights at every five epochs
    period=5,  # Save weights every five epochs
    verbose=1  # Display a message when the weights are saved
)


fit_nn = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 9s 3ms/step - loss: 0.6136 - accuracy: 0.6820
Epoch 2/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5768 - accuracy: 0.7177
Epoch 3/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5682 - accuracy: 0.7223
Epoch 4/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5640 - accuracy: 0.7279
Epoch 5/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5617 - accuracy: 0.7290
Epoch 6/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5604 - accuracy: 0.7287
Epoch 7/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5596 - accuracy: 0.7283
Epoch 8/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5590 - accuracy: 0.7287
Epoch 9/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5585 - accuracy: 0.7275
Epoch 10/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5580 - accuracy: 0.7289

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5602 - accuracy: 0.7249 - 611ms/epoch - 2ms/step
Loss: 0.5601572394371033, Accuracy: 0.7248979806900024


In [27]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
